In [1]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
     
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


In [2]:
# b = chat.predict("How many planets are there?")
# b

template = PromptTemplate.from_template("{country_a}와 {country_b}사이의 거리는 어떻게되나요?")
prompt = template.format(country_a="맥시코", country_b="인도")
chat.predict(prompt)

맥시코와 인도 사이의 거리는 직선 거리로 약 15,000 킬로미터입니다. 그러나 실제로 여행하는 경우에는 항공로나 해상로를 이용하므로 실제 거리는 이보다 더 길어질 수 있습니다.

'맥시코와 인도 사이의 거리는 직선 거리로 약 15,000 킬로미터입니다. 그러나 실제로 여행하는 경우에는 항공로나 해상로를 이용하므로 실제 거리는 이보다 더 길어질 수 있습니다.'

In [3]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가입니다. 그리고 당신은 {language}로만 대답을 합니다."),
    ("ai", "안녕하세요 제 이름은 {name}입니다."),
    ("human", "{country_a}와 {country_b}사이의 거리는 어떻게되나요? 그리고 당신의 이름은 무었입니까?")
])
prompt = template.format_messages(language="일본어", name="길동", country_a="일본", country_b="한국")
chat.predict_messages(prompt)

안녕하세요! 일본과 한국 사이의 거리는 대략 900km 정도입니다. 저의 이름은 길동입니다.

AIMessageChunk(content='안녕하세요! 일본과 한국 사이의 거리는 대략 900km 정도입니다. 저의 이름은 길동입니다.')

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse(" a ,  b ,  c ,   d")

['a', 'b', 'c', 'd']

In [5]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}")
])
# prompt = template.format_messages(
#     max_items=10,
#     question="What are the planets"
# )
prompt = template.format_messages(
    max_items=10,
    question="What are the colors"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)


red, orange, yellow, green, blue, indigo, violet, black, white, gray

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [6]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question":"What are the pokemons?"
})

pikachu, charizard, bulbasaur, squirtle, jigglypuff

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [7]:
chef_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisins with easy to find ingredients."),
        ("human","I want to cook {cuisine} food"),
    ]
)
chef_chain = chef_prompt | chat
# chef_chain.invoke({
#     "cuisine":"korean"
# })

In [8]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find a alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat



In [9]:
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine":"korean"
})

Great choice! Korean cuisine is known for its bold flavors and unique combinations. Here's a recipe for a classic Korean dish called Bibimbap:

Ingredients:
- 2 cups cooked rice
- 1 cup thinly sliced beef (you can also use chicken or tofu as a vegetarian option)
- 1 cup julienned carrots
- 1 cup julienned cucumber
- 1 cup bean sprouts
- 1 cup spinach
- 4 eggs
- 4 tablespoons gochujang (Korean chili paste)
- 2 tablespoons soy sauce
- 2 tablespoons sesame oil
- 2 cloves garlic, minced
- Salt and pepper to taste
- Sesame seeds for garnish

Instructions:
1. Marinate the beef (or chicken/tofu) with soy sauce, minced garlic, sesame oil, salt, and pepper. Let it sit for at least 15 minutes.
2. In a large skillet, heat some oil over medium heat. Stir-fry the marinated beef (or chicken/tofu) until cooked through. Set aside.
3. In the same skillet, add a little more oil and stir-fry the carrots until slightly softened. Remove from the skillet and set aside.
4. Repeat the same process with the cu

AIMessageChunk(content="For the vegetarian version of Bibimbap, you can replace the thinly sliced beef with tofu. Here's how you can prepare the tofu:\n\nIngredients:\n- 1 cup firm tofu, pressed and drained\n- 2 tablespoons soy sauce\n- 1 tablespoon sesame oil\n- 1 clove garlic, minced\n- Salt and pepper to taste\n\nInstructions:\n1. Cut the tofu into thin slices or cubes.\n2. In a bowl, combine soy sauce, sesame oil, minced garlic, salt, and pepper.\n3. Add the tofu to the marinade and let it sit for at least 15 minutes to absorb the flavors.\n4. Heat a non-stick skillet over medium heat and add a little oil.\n5. Cook the marinated tofu in the skillet until it is browned and crispy on all sides.\n6. Remove the tofu from the skillet and set aside.\n\nContinue with the rest of the recipe as mentioned, substituting the cooked tofu for the beef in step 2 and arranging it on top of the rice in step 6.\n\nEnjoy your vegetarian Bibimbap!")

In [10]:
# 4.1 FewShotPromptTemplate

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")

# t=PromptTemplate(template="What is the capital of {country}", input_variables=["country"])
# t.format(country="japan")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat
chain.invoke({"country": "Germany"})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [11]:
# 4.2 FewShotChatMessagePromptTemplate
examples = [
    {
        "country": "France",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "country": "Italy",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "country": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give shorts answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat
chain.invoke({"country": "Korea"})

I know this:
There are two Koreas - North Korea and South Korea.
Capital of North Korea: Pyongyang
Capital of South Korea: Seoul
Languages: Korean (North Korean dialect and South Korean dialect)
Food: Kimchi, Bibimbap, Bulgogi
Currency: North Korean Won (North Korea), South Korean Won (South Korea)

AIMessageChunk(content='I know this:\nThere are two Koreas - North Korea and South Korea.\nCapital of North Korea: Pyongyang\nCapital of South Korea: Seoul\nLanguages: Korean (North Korean dialect and South Korean dialect)\nFood: Kimchi, Bibimbap, Bulgogi\nCurrency: North Korean Won (North Korea), South Korean Won (South Korea)')

In [12]:
# 4.3 LengthBasedExampleSelector

from typing import Any, Dict


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

class RandomExampleSelector(BaseExampleSelector):
     
     def __init__(self, examples):
          self.examples = examples

     def select_examples(self, input_variables):
          from random import choice
          return [choice(self.examples)]
     
     def add_example(self, example):
          self.examples.append(example)
     
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=10,
# )
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
prompt.format(country="Korea")

# chain = prompt | chat
# chain.invoke({"country": "Germany"})

'\n    Human: What do you know about France?\n    AI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\nHuman: What do you know about Korea?'

In [16]:
# 4.4 Serialization and Composition
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

'What is the captial of Germany'

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
""")
example = PromptTemplate.from_template(
    """
    This is and example of how you talk:

    Human: {example_question}
    You: {example_answer}
""")
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You: 
"""
)
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
""")

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts,)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat
chain.invoke({
    "character": "Pirate",
    "example_question": "What is your location?",
    "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
    "question": "What is your fav food?",
})


Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy fried fish and salty chips be enough to make me heart sing like a sea shanty! What about ye, matey? What be yer favorite grub?

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy fried fish and salty chips be enough to make me heart sing like a sea shanty! What about ye, matey? What be yer favorite grub?")

In [30]:
# 4.5 Chaching
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)
chat.predict("How do you make italian pasta?")



[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [33.85s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a bit more flour.\

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a bit more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into smaller